In [11]:
import pandas as pd

In [23]:
df = pd.read_csv("../data/synthetic_right.csv")

In [24]:
df.describe()

,Unnamed: 0,elapsed,dice3d_0.1,dice3d_0.2285714285714286,dice3d_0.3571428571428572,dice3d_0.48571428571428577,dice3d_0.6142857142857143,dice3d_0.7428571428571429,dice3d_0.8714285714285716,dice3d_1.0,backproj_dice3d_0
count,4.0,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.0,4.0,4.000000
mean,0.0,6.363407,0.311988,0.600306,0.471550,0.241981,0.066407,0.001953,0.0,0.0,0.087014
std,0.0,0.321766,0.356270,0.039687,0.113130,0.103040,0.057029,0.003906,0.0,0.0,0.032731
min,0.0,5.973029,0.003849,0.542683,0.312834,0.112500,0.009967,0.000000,0.0,0.0,0.040548
25%,0.0,6.224465,0.003969,0.594766,0.429162,0.182301,0.023718,0.000000,0.0,0.0,0.080998
50%,0.0,6.363608,0.299877,0.612470,0.508905,0.265145,0.061977,0.000000,0.0,0.0,0.095035
75%,0.0,6.502550,0.607896,0.618011,0.551293,0.324826,0.104666,0.001953,0.0,0.0,0.101050
max,0.0,6.753383,0.644351,0.633602,0.555556,0.325134,0.131707,0.007812,0.0,0.0,0.117436
